In [1]:
import torch

import requests
import json

import math
from collections import Counter

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

from sentence_transformers import SentenceTransformer

In [2]:
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#test set 1 = season 8

with open('sets/test_set1.json') as f:
    test_set1 = json.load(f)

with open('sets/train_set1.json') as f:
    train_set1 = json.load(f)

################################################################

#test set 2 = 10% of each season

with open('sets/test_set2.json') as f:
    test_set2 = json.load(f)


In [3]:
def tf_idf_calculator(lines):    
    # Compute the term frequency of each word in each sentence
    tf = []
    
    for sentence in lines:
        tf_sentence = {}
        for word in sentence.split():
            tf_sentence[word] = tf_sentence.get(word, 0) + 1
        tf.append(tf_sentence)

    # Compute the inverse document frequency of each word
    idf = {}
    for sentence in lines:
        for word in sentence.split():
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = math.log(len(lines) / idf[word])

    # Compute the tf-idf of each sentence
    tf_idf = []
    for i, sentence in enumerate(lines):
        tf_idf_sentence = 0
        for word in tf[i]:
            tf_idf_sentence += tf[i][word] * idf[word]
        tf_idf.append(tf_idf_sentence)
    
    return tf_idf

In [13]:
#substitue train_set1/2

values = []


for character in characters:

    lines_withcontext = []
    lines = []

    for idx, utterance in enumerate(train_set1):
        if utterance['speakers'] != [] and character in utterance['speakers'] and idx !=0:
            lines_withcontext.append([utterance['transcript'], train_set1[idx-1]['transcript']])
            lines.append(utterance['transcript'])
        elif idx ==0:
            lines_withcontext.append([utterance['transcript'], ""])
            lines.append(utterance['transcript'])
    
    
    tf_idf = tf_idf_calculator(lines)

    final = []

    #format: final = [...,[sentence, tf-idf score],...]
    for i,sentence in enumerate(lines):
        cell = []
        cell= [tf_idf[i], sentence]
        final.append(cell)
    
    # lower -> higher     
    sorted_list = sorted(final)

    l = []

    #get only the strings
    for elem in sorted_list:
        l.append(elem[1])

    #remove duplicates
    l = list( dict.fromkeys(l) )

    #change x for number of lines 
    values = [5,10,20,50,100,150,200,500,1000,2000,5000, 6000]

    for x in values:

        higher = l[-x:]
        lower = l[:x]

        #substitute embeddings1/2

        f = open("embeddingscontext1/" + character + str(x)+ ".txt", "w")
        for line in higher:
            index = lines.index(line)

            for utterance in lines_withcontext[index]:
                f.write(utterance +"\n")
        f.close()

        f = open("embeddingscontext1/" + "others" + str(x)+ ".txt", "w")
        for line in lower:
            index = lines.index(line)
            
            for utterance in lines_withcontext[index]:
                f.write(utterance +"\n")
        f.close()




In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def compute_embedding(previous_sentence,current_sentence):
        
    
    embedding_previous = model.encode(previous_sentence)
    embedding_current = model.encode(current_sentence)

    embedding = (embedding_previous + embedding_current)/2

    return embedding

In [7]:
sentences = []

for character in characters:
    f = open("embeddingscontext1/" + character +"5"+ ".txt", "r")
    lines = f.readlines()
    sentences.append(lines)


i=0
embeddings = {}
for character in characters:
    embeddings[character] = []
    for idx, utterance in enumerate(sentences[i]):
        if (idx % 2)==0:
            embeddings[character].append(compute_embedding(sentences[i][idx+1], utterance))

    i+=1

#print(embeddings)

for character in characters:

    sum = 0
    for embedding in embeddings[character]:
        sum += embedding

    
    embeddings[character] = sum


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
def compute_similarity(vector):

    angles = {}

    for character in characters:
        angles[character] = cosine_similarity(vector.reshape(1,-1), embeddings[character].reshape(1,-1))[0][0]

    
    
    #Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity

    character = [i for i in angles if angles[i]==max(angles.values())]


    return character[0]

In [8]:
predicted = []
real = []

correct = 0
total = 0

for idx, utterance in enumerate(test_set1):
    if len(utterance['speakers']) == 1:

        total += 1
        if idx == 0:
            line_embed = compute_embedding("", utterance['transcript'])
        else:
            line_embed = compute_embedding(test_set1[idx-1]['transcript'], utterance['transcript'])
    
        pred = compute_similarity(line_embed)

        predicted.append(pred)
        real.append(utterance['speakers'][0])

        if pred == utterance['speakers'][0]:
            correct+=1
        elif pred == "others" and utterance['speakers'][0] not in ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']:
            correct+=1

                    
print(accuracy_score(real, predicted)*100)

accuracy = (float(correct)/total)*100

print("accuracy: ", accuracy)

#print("f1 score: ", f1_score(real, predicted, average='macro'))

print(len(characters))

17.823216873289326
accuracy:  17.96812107551119
7
